In [ ]:
import sys
import os
import json
import pandas as pd

sys.path.append(os.path.abspath("N:\\CancerEpidem\\BrBreakthrough\\DeliveryProcess\\Schema_and_Derivation_utils\\Questionnaire\\R0\\scripts"))
from common_utils import load_and_pivot_data, load_schema, process_data, save_output, mask_pii, get_config, validate_data
from schema_utils import build_variable_mapping, extract_constraints, extract_var_types
from processing_utils import process_data, save_change_tracking
from restructure_utils import restructure_physical_dev
import CleaningRules as cr
import NestedVariables as nv

sys.path.append(os.path.abspath("N:\\CancerEpidem\\BrBreakthrough\\DeliveryProcess\Schema_and_Derivation_utils"))
from utilities import connect_DB, createLogger, read_data
from config import Delivery_log_path, test_server, r0_json_path, out_json_path

In [2]:
# Initialize
config = get_config()
logger = createLogger('PhysicalDevelopment', config['Delivery_log_path'])
question_range = 'BETWEEN 101 AND 187'

In [ ]:
# Load data
pivoted, dfPII = load_and_pivot_data(question_range, logger)

In [ ]:
pivoted = pivoted.fillna('').reset_index()
pivotedDict = pivoted.set_index('StudyID').to_dict('index')

In [ ]:
schema = load_schema('PhysicalDevelopment_JSON')
schema_props = schema["additionalProperties"]["properties"]

In [ ]:
variable_mapping = build_variable_mapping(schema)
constraint_map = extract_constraints(schema_props)
var_type_map = extract_var_types(schema_props)

Extracted constraints for all fields
Extracted variable types for all fields


In [ ]:
# Process data
processed_data, change_tracking = process_data(
    pivotedDict,
    variable_mapping,
    var_type_map,
    constraint_map,
    cr.newValMap
)

In [ ]:
processed_data[100042]

In [ ]:
json_data = restructure_physical_dev(processed_data, schema, variable_mapping)

In [ ]:
json_data[100042]

In [ ]:
validate_data(json_data, schema)

JSON data is valid.


In [ ]:
# Remove PII
json_data = mask_pii(json_data, dfPII)

In [13]:
# Save output
save_output(json_data, 'Output_PhysicalDevelopment', logger)

2025-06-13 11:45:27 - INFO: Saved output to N:\CancerEpidem\BrBreakthrough\DeliveryProcess\Data_Output_Testing\Output_PhysicalDevelopment.json


Output saved: N:\CancerEpidem\BrBreakthrough\DeliveryProcess\Data_Output_Testing\Output_PhysicalDevelopment.json
